<a href="https://colab.research.google.com/github/slatej1/Everside/blob/main/Parse_investment_template_to_deal_cloud_file.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import openpyxl

In [14]:

# UPDATE INPUT FILE NAME
# Read the data from the source file
source_file_path = 'Investment Template - Oxer.xlsx'
data = pd.read_excel(source_file_path, sheet_name='eFile', skiprows = 7)  # Adjust sheet_name as needed

# print(data.head())

In [15]:
template_file_path = 'deal_cloud_dataset_template_for_python.xlsx'
workbook = openpyxl.load_workbook(template_file_path)
worksheet = workbook.active
# UPDATE OUTPUT FILE NAME
output_file_path = 'output_file_Oxer-v1.xlsx'

In [16]:
column_mapping = {
    'Company Name': 'E',
    'Industry': 'I',
    'Date of Investment Memo': 'D',
    'Revenue': ['BA', 'BP'],
    'Adj. EBITDA ': ['BB', 'BQ'],
    'Implied EV': ['AY', 'BN'],
    'Description': 'DF',
    'Sourcing': 'K',
    'Sourcing Description': 'L',
    'Investment': 'V',
    'Total Investment': 'S',
    'Senior Debt.1': ['AS', 'BE'],
    'Sr. Subordinated debt.1': ['AU', 'BG'],
    'Cash': 'AK',
    'Dividend/PIK': 'AL',
    'Cash.1': 'AP',
    'Dividend/PIK.1': 'AQ',
    'Ownership.1': 'AR',
    'Tenor': 'AO',
    'CombinedColumn': {'sources': ['Cash', 'Dividend/PIK'], 'target': 'AM'} ,
    'CombinedColumn': {'sources': ['Senior Debt.1', 'Sr. Subordinated debt.1'], 'target': 'Q'} ,
    'SubtractedColumn': {'sources': ['Total Investment', 'Senior Debt.1', 'Sr. Subordinated debt.1'], 'target': 'P'}
}

In [17]:
def col_letter_to_num(letter):
    num = 0
    for char in letter:
        num = num * 26 + (ord(char.upper()) - ord('A') + 1)
    return num

In [18]:
start_row = 2

# Set the first three columns to "Pipeline"
for i in range(len(data)):
    for col_num in range(1, 4):
        worksheet.cell(row=start_row + i, column=col_num, value="Pipeline")

# Write data to the template based on the column mapping
for i, row in data.iterrows():
    for source_col, target_cols in column_mapping.items():
        if isinstance(target_cols, list):
            # Handle mapping a single source column to multiple target columns
            for target_col in target_cols:
                value = row[source_col]
                col_num = col_letter_to_num(target_col)  # Convert column letter to number
                worksheet.cell(row=start_row + i, column=col_num, value=value)
        elif isinstance(target_cols, dict) and 'sources' in target_cols:
            # Handle combined or subtracted columns
            sources = target_cols['sources']
            target_col_letter = target_cols['target']
            if source_col.startswith('Subtracted'):
                # Subtract the sum of specified source columns
                value = row[sources[0]] - sum(row[source] for source in sources[1:])
            else:
                # Combine specified columns
                value = sum(row[source] for source in sources)
            col_num = col_letter_to_num(target_col_letter)  # Convert column letter to number
            worksheet.cell(row=start_row + i, column=col_num, value=value)
        else:
            # Single column mapping
            value = row[source_col]
            col_num = col_letter_to_num(target_cols)  # Convert column letter to number
            worksheet.cell(row=start_row + i, column=col_num, value=value)

In [19]:
# Save the updated template
workbook.save(output_file_path)